In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Получим данные Мос.биржи

In [49]:
df = pd.read_excel('../data/EuqityMOEXReturns.xlsx', sheet_name='D')
df = df.drop(columns=df.columns[df.isna().sum() > len(df) / 2])
df = df.dropna()
df = df.rename(columns={'Unnamed: 0': 'date'})
df['date'] = df['date'].dt.date
df = df.sort_values(by='date').reset_index()
df_stocks = df.drop(['date'], axis=1)
df.shape

/tmp/ipykernel_6806/869702920.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel('../data/EuqityMOEXReturns.xlsx', sheet_name='D')


(899, 69)

## Посмотрим на поведение базовой цены

In [50]:
for _ in range(5):
    sample = df_stocks.sample(axis='columns')
    plt.plot(df['date'], sample)
    plt.title(f"График цены у {sample.columns[0]}")
    plt.xlabel('День')
    plt.ylabel('Значение цены')
    plt.show()

## Посмотрим на поведение среднего

In [51]:
for _ in range(5):
    sample = df_stocks.sample(axis='columns')
    plt.plot(df['date'], sample.rolling(len(df),min_periods=2).mean(), label=sample.columns[0])
    name = df_stocks.sample(axis='columns').columns[0]
    plt.xlabel('День')
    plt.ylabel('Значение среднего')

plt.title(f"Аккумулированное среднее")
plt.show()

## Посмотрим на поведение дисперсий

In [52]:
for _ in range(5):
    sample = df_stocks.sample(axis='columns')
    plt.plot(df['date'], sample.rolling(len(df),min_periods=2).std())
    plt.title(f"Аккумулированная дисперсия у {sample.columns[0]}")
    name = df_stocks.sample(axis='columns').columns[0]
    plt.xlabel('День')
    plt.ylabel('Значение дисперсии')
    plt.show()

In [53]:
for _ in range(5):
    sample = df_stocks.sample(axis='columns')
    plt.plot(df['date'], sample.rolling(len(df),min_periods=2).std(), label=sample.columns[0])
    name = df_stocks.sample(axis='columns').columns[0]
    plt.xlabel('День')
    plt.ylabel('Значение дисперсии')

plt.title(f"Аккумулированная дисперсия")
plt.show()

### Видно, что дисперсия довольно явно скачет. Из чего можно сделать вывод, что разладка случается. Теперь проделаем анализ окном

In [54]:
for _ in range(5):
    sample = df_stocks.sample(axis='columns')
    plt.plot(df['date'], sample.rolling(100).mean(), label=sample.columns[0])
    plt.xlabel('День')
    plt.ylabel('Значение среднего')

plt.title(f"Оконное среднее")
plt.show()

In [55]:
for _ in range(5):
    sample = df_stocks.sample(axis='columns')
    plt.plot(df['date'], sample.rolling(100).std(), label=sample.columns[0])
    plt.xlabel('День')
    plt.ylabel('Значение дисперсии')

plt.legend()
plt.title(f"Оконная дисперсия")
plt.show()

In [56]:
for _ in range(2):
    sample = df_stocks.sample(axis='columns')
    plt.plot(df['date'], sample.rolling(3).std(), label=sample.columns[0])
    plt.xlabel('День')
    plt.ylabel('Значение дисперсии')

plt.title(f"Оконная дисперсия")
plt.show()

Итак, видно, что на графике есть моменты когда дисперсия становится как аномально большой, так и аномально маленькой
Также видно, что переходы в дисперсии почти мгновенные. Можно попробовать смоделировать ряд и посмотреть, как будут вести себя алгоритмы

## Сперва проверим, как ведут себя тесты в базовом случае

In [57]:
import numpy as np

mean = [0, 0, 0]
covariance_matrix = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
num_samples = 1255

multivariate_noise = np.random.multivariate_normal(mean, covariance_matrix, num_samples)

In [58]:
from change_point.methods.Quantile import create_sequence_of_quantiles

In [59]:
quantile = [*create_sequence_of_quantiles(multivariate_noise, border=0.9)]
plt.plot(np.cumsum(quantile), label='test')
plt.plot([(1-0.955)*x for x in range(num_samples)], label='ideal')
plt.legend()
plt.show()

Видно, что уровень лжесрабатываний почти полностью совпадает с идеальным

In [60]:
from change_point.methods.CuSum import create_sequence_of_cusums
from tqdm import tqdm

In [61]:
cusum_3 = [*create_sequence_of_cusums(multivariate_noise, border=3.)]
cusum_2 = [*create_sequence_of_cusums(multivariate_noise, border=2.)]
cusum_1 = [*create_sequence_of_cusums(multivariate_noise, border=1.)]

100%|██████████| 1250/1250 [00:10<00:00, 116.38it/s]


In [62]:
plt.plot(cusum_1, label='test_1')
plt.plot(cusum_2, label='test_2')
plt.plot(cusum_3, label='test_3')
plt.legend()
plt.show()

In [63]:
from change_point.methods.LambdaMax import create_sequence_of_norms

In [64]:
lambda_30 = [*create_sequence_of_norms(multivariate_noise, 30)]
lambda_20 = [*create_sequence_of_norms(multivariate_noise, 20)]
lambda_10 = [*create_sequence_of_norms(multivariate_noise, 10)]

100%|██████████| 1245/1245 [00:00<00:00, 5382.03it/s]


In [65]:
plt.plot(lambda_10, label='test_10')
plt.plot(lambda_20, label='test_20')
plt.plot(lambda_30, label='test_30')
plt.legend()
plt.show()

In [66]:
from change_point.methods.COCPD import get_sequence_of_COCPD

In [67]:
cocpd = [*get_sequence_of_COCPD(multivariate_noise[:20])]

100%|██████████| 15/15 [02:42<00:00, 10.86s/it]


In [68]:
plt.plot(range(8, 20), cocpd[3:], label='COCPD')
plt.legend()
plt.show()